In [1]:
import geopandas as gpd
from shapely.geometry import Point, LineString, MultiPoint, MultiLineString
from shapely.ops import nearest_points
import pandas as pd
import glob

gdf_distritos = gpd.read_file('../download/SIRGAS_SHP_distrito/SIRGAS_SHP_distrito_polygon.shp')
#print(gdf_distritos[["ds_codigo", "ds_nome"]])

In [2]:
gdf_vias = gpd.read_file('../download/SIRGAS_SHP_logradouronbl/SIRGAS_SHP_logradouronbl.shp')

In [84]:
gdf_quadras = gpd.read_file('../download/SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp')

In [3]:
def centroide_por_segmento(linha):
    segmentos = list(map(LineString, zip(linha.coords[:-1], linha.coords[1:])))
    centroides = list(map(lambda x: x.centroid, segmentos))
    return MultiPoint(centroides)

In [4]:
def segmentos(linha):
    segmentos = list(map(LineString, zip(linha.coords[:-1], linha.coords[1:])))
    # centroides = list(map(lambda x: x.centroid, segmentos))
    return MultiLineString(segmentos)

In [82]:
## Para cada distrito
folder = '../download/lotes'
for path in glob.iglob(f"{folder}/*"):
    cod_distrito = path.replace(f"{folder}/SIRGAS_SHP_LOTES_", '')[0:2]
    gdf_distrito = gdf_distritos[gdf_distritos.ds_codigo == cod_distrito]

    ## Recorta NBL apenas do Distrito considerando um buffer de 20m
    gdf_viario = gpd.clip(gdf_vias, gdf_distrito.buffer(30))

    ## Dissolve a NBL por CODLOG
    gdf_viario = gdf_viario.dissolve(by='lg_codlog').reset_index()

    ## Abre o arquivo de lotes do distrito
    gdf_lotes = gpd.read_file(f"zip://{path}!{path.replace(folder, '').replace('.zip', '')[1:]}")

    ## Dissolve o arquivo de lotes por Setor, Quadra, Tipo e talvez Subquadra
    gdf_lotes_dissolved = gdf_lotes.dissolve(by=['lo_setor', 'lo_quadra', 'lo_tp_quad']).reset_index()
    gdf_faces = gpd.GeoDataFrame([], geometry=gdf_lotes_dissolved.boundary.explode().droplevel(0).reset_index().drop(columns='index')[0])

    ## Calcula-se o centroide de cada segmento de linha de cada quadra
    gdf_pontos_faces = gpd.GeoDataFrame(geometry=gdf_faces.geometry.apply(lambda x: centroide_por_segmento(x)))

    ## Cria um buffer de alguns centimetros para o lote para certificar que o ponto esteja inserido nele
    gdf_lotes_buffered = gpd.GeoDataFrame(gdf_lotes, geometry=gdf_lotes.buffer(.05))

    ## Relaciona cada centroide de segmento de linha ao lote
    gdf_pontos_lotes = gpd.sjoin(gdf_pontos_faces.reset_index().explode().droplevel(0).drop(columns='index'), gdf_lotes_buffered, how='left')

    ## Cria um viário unificado para projetar os pontos mais próximos
    gdf_viario_unified = gdf_viario.unary_union
    
    ## Conecta-se o centroide de cada segmento de linha ao segmento de logradouro mais próximo
    gdf_pontos_lotes_copy = gdf_pontos_lotes.copy(deep=True)
    gdf_conexoes = gpd.GeoDataFrame(gdf_pontos_lotes_copy, \
        geometry=gdf_pontos_lotes_copy.geometry.apply(lambda x: \
            LineString(nearest_points(x, gdf_viario_unified))))

    ## Cria Buffer de 50cm do viario
    gdf_viario_buffered = gpd.GeoDataFrame(gdf_viario, geometry=gdf_viario.buffer(.10))

    ## ajuste de indice
    gdf_conexoes = gdf_conexoes.drop(columns=['index_right'])

    ## Cria conexão com a via
    gdf_conexoes_relacionadas = gpd.sjoin(gdf_conexoes, \
        gdf_viario_buffered[['lg_codlog', 'geometry']], how='left', op='intersects')

    ## Relaciona os pontos ao CODLOG
    gdf_pontos_lotes_buffered = gpd.GeoDataFrame(gdf_pontos_lotes, geometry=gdf_pontos_lotes.buffer(0.01))
    gdf_pontos_lote_buffered_relacionados = gpd.sjoin(gdf_pontos_lotes_buffered.drop(columns=['index_right']), gdf_conexoes_relacionadas[['lg_codlog', 'geometry']], how='left')

    ## Segmentos de faces de lotes
    gdf_faces_segmentos = gpd.GeoDataFrame(geometry=gdf_faces.geometry.apply(lambda x: segmentos(x)))
    gdf_faces_segmentos = gdf_faces_segmentos.explode().droplevel(0).reset_index().drop(columns=['index'])  

    # ## Relaciona-se essa conexão criada ao CODLOG e Lote
    gdf_testadas = gpd.sjoin(gdf_faces_segmentos, gdf_pontos_lote_buffered_relacionados.drop(columns=['index_right']), how='left', op='intersects')

    ## TODO
    ## Remover "sujeira" de linhas internas na quadra. Sobrepondo a geometria das quadras fiscais

    ## Dissolve-se os segmentos de linha das faces dos lotes em testadas por CODLOG agrupadas por Lote fiscal
    gdf_testadas_dissolved = gdf_testadas.dissolve(by=['lo_setor', 'lo_quadra', 'lo_lote', 'lo_condomi', 'lo_tp_lote', 'lg_codlog'], as_index=False)

    ## Ajustando o SRC
    gdf_testadas_dissolved = gdf_testadas_dissolved.set_crs('epsg:31983')

    ## Exporta os resultados para um arquivo contendo as testadas dos lotes do distrito selecionado
    distrito_prefix = path.replace(f"{folder}/SIRGAS_SHP_LOTES_", '')[0:].replace('.zip', '').replace('_', '-').lower()
    gdf_testadas_dissolved.to_file(f"../resultados/{distrito_prefix}-testadas.gpkg", driver='GPKG')

    ## Grava Log
    print(distrito_prefix)


90-vila-mariana
